Scraping from http://fbref.com

Data:
- outfield players stats
- goalkeepers stats
- team stats (of and against)

Original code by parth1902:
https://github.com/parth1902/Scrape-FBref-data

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import sys, getopt
import csv


In [14]:
#standard(stats)
stats = ["player","nationality","position","team","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]
stats3 = ["players_used","possession","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"] 
#goalkeeping(keepers)
keepers = ["player","nationality","position","team","age","birth_year","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","gk_shots_on_target_against","gk_saves","gk_save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","gk_clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]
keepers3 = ["players_used","gk_games","gk_games_starts","gk_minutes","gk_goals_against","gk_goals_against_per90","gk_shots_on_target_against","gk_saves","gk_save_pct","gk_wins","gk_ties","gk_losses","gk_clean_sheets","gk_clean_sheets_pct","gk_pens_att","gk_pens_allowed","gk_pens_saved","gk_pens_missed"]
#advance goalkeeping(keepersadv)
keepersadv = ["player","nationality","position","team","age","birth_year","minutes_90s","gk_goals_against","gk_pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","gk_psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]
keepersadv2 = ["minutes_90s","gk_goals_against","gk_pens_allowed","gk_free_kick_goals_against","gk_corner_kick_goals_against","gk_own_goals_against","gk_psxg","gk_psnpxg_per_shot_on_target_against","gk_psxg_net","gk_psxg_net_per90","gk_passes_completed_launched","gk_passes_launched","gk_passes_pct_launched","gk_passes","gk_passes_throws","gk_pct_passes_launched","gk_passes_length_avg","gk_goal_kicks","gk_pct_goal_kicks_launched","gk_goal_kick_length_avg","gk_crosses","gk_crosses_stopped","gk_crosses_stopped_pct","gk_def_actions_outside_pen_area","gk_def_actions_outside_pen_area_per90","gk_avg_distance_def_actions"]
#shooting(shooting)
shooting = ["player","nationality","position","team","age","birth_year","minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting2 = ["minutes_90s","goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
shooting3 = ["goals","pens_made","pens_att","shots_total","shots_on_target","shots_free_kicks","shots_on_target_pct","shots_total_per90","shots_on_target_per90","goals_per_shot","goals_per_shot_on_target","xg","npxg","npxg_per_shot","xg_net","npxg_net"]
#passing(passing)
passing = ["player","nationality","position","team","age","birth_year","minutes_90s","passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
passing2 = ["passes_completed","passes","passes_pct","passes_total_distance","passes_progressive_distance","passes_completed_short","passes_short","passes_pct_short","passes_completed_medium","passes_medium","passes_pct_medium","passes_completed_long","passes_long","passes_pct_long","assists","xa","xa_net","assisted_shots","passes_into_final_third","passes_into_penalty_area","crosses_into_penalty_area","progressive_passes"]
#passtypes(passing_types)
passing_types = ["player","nationality","position","team","age","birth_year","minutes_90s","passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
passing_types2 = ["passes","passes_live","passes_dead","passes_free_kicks","through_balls","passes_pressure","passes_switches","crosses","corner_kicks","corner_kicks_in","corner_kicks_out","corner_kicks_straight","passes_ground","passes_low","passes_high","passes_left_foot","passes_right_foot","passes_head","throw_ins","passes_other_body","passes_completed","passes_offsides","passes_oob","passes_intercepted","passes_blocked"]
#goal and shot creation(gca)
gca = ["player","nationality","position","team","age","birth_year","minutes_90s","sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
gca2 = ["sca","sca_per90","sca_passes_live","sca_passes_dead","sca_dribbles","sca_shots","sca_fouled","gca","gca_per90","gca_passes_live","gca_passes_dead","gca_dribbles","gca_shots","gca_fouled","gca_defense"]
#defensive actions(defense)
defense = ["player","nationality","position","team","age","birth_year","minutes_90s","tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
defense2 = ["tackles","tackles_won","tackles_def_3rd","tackles_mid_3rd","tackles_att_3rd","dribble_tackles","dribbles_vs","dribble_tackles_pct","dribbled_past","pressures","pressure_regains","pressure_regain_pct","pressures_def_3rd","pressures_mid_3rd","pressures_att_3rd","blocks","blocked_shots","blocked_shots_saves","blocked_passes","interceptions","clearances","errors"]
#possession(possession)
possession = ["player","nationality","position","team","age","birth_year","minutes_90s","touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
possession2 = ["touches","touches_def_pen_area","touches_def_3rd","touches_mid_3rd","touches_att_3rd","touches_att_pen_area","touches_live_ball","dribbles_completed","dribbles","dribbles_completed_pct","players_dribbled_past","nutmegs","carries","carry_distance","carry_progressive_distance","progressive_carries","carries_into_final_third","carries_into_penalty_area","pass_targets","passes_received","passes_received_pct","miscontrols","dispossessed"]
#playingtime(playingtime)
playingtime = ["player","nationality","position","team","age","birth_year","minutes_90s","games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
playingtime2 = ["games","minutes","minutes_per_game","minutes_pct","games_starts","minutes_per_start","games_subs","minutes_per_sub","unused_subs","points_per_match","on_goals_for","on_goals_against","plus_minus","plus_minus_per90","plus_minus_wowy","on_xg_for","on_xg_against","xg_plus_minus","xg_plus_minus_per90","xg_plus_minus_wowy"]
#miscallaneous(misc)
misc = ["player","nationality","position","team","age","birth_year","minutes_90s","cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]
misc2 = ["cards_yellow","cards_red","cards_yellow_red","fouls","fouled","offsides","crosses","interceptions","tackles_won","pens_won","pens_conceded","own_goals","ball_recoveries","aerials_won","aerials_lost","aerials_won_pct"]

def get_tables(url,text):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

    res = requests.get(url, headers = headers)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    team_table = all_tables[0]
    team_vs_table = all_tables[1]
    player_table = all_tables[2]
    if text == 'for':
      return player_table, team_table
    if text == 'vs':
      return player_table, team_vs_table

def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                if(cell == None):
                  continue
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))                  
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)
    return df_player

def get_frame_team(features, team_table):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    for row in rows_squad:
        if(row.find('th',{"scope":"row"}) != None):
            name = row.find('th',{"data-stat":"team"}).text.strip().encode().decode("utf-8")
            if 'team' in pre_df_squad:
                pre_df_squad['team'].append(name)
            else:
                pre_df_squad['team'] = [name]
            for f in features_wanted_squad:
                cell = row.find("td",{"data-stat": f})
                if(cell == None):
                  continue
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    return df_squad

def get_frame_team_2(features, team_table):
    pre_df_squad = dict()
    #Note: features does not contain squad name, it requires special treatment
    features_wanted_squad = features
    rows_squad = team_table.find_all('tr')
    for row in rows_squad:
        if(row.find('th',{"scope":"row"}) != None):
            name = row.find('th',{"data-stat":"squad"}).text.strip().encode().decode("utf-8")
            if 'squad' in pre_df_squad:
                pre_df_squad['squad'].append(name)
            else:
                pre_df_squad['squad'] = [name]
            for f in features_wanted_squad:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')&(f!='team')):
                    text = float(text.replace(',',''))
                    print(f)
                if f in pre_df_squad:
                    pre_df_squad[f].append(text)
                else:
                    pre_df_squad[f] = [text]
    df_squad = pd.DataFrame.from_dict(pre_df_squad)
    return df_squad

def frame_for_category(category,top,end,features):
    url = (top + category + end)
    player_table, team_table = get_tables(url,'for')
    df_player = get_frame(features, player_table)
    return df_player

def frame_for_category_team(category,top,end,features,text):
    url = (top + category + end)
    player_table, team_table = get_tables(url,text)
    df_team = get_frame_team(features, team_table)
    return df_team




#Function to get the player data for outfield player, includes all categories - standard stats, shooting
#passing, passing types, goal and shot creation, defensive actions, possession, and miscallaneous
def get_outfield_data(top, end):
    df1 = frame_for_category('stats',top,end,stats)
    df2 = frame_for_category('shooting',top,end,shooting2)
    df3 = frame_for_category('passing',top,end,passing2)
    df4 = frame_for_category('passing_types',top,end,passing_types2)
    df5 = frame_for_category('gca',top,end,gca2)
    df6 = frame_for_category('defense',top,end,defense2)
    df7 = frame_for_category('possession',top,end,possession2)
    df8 = frame_for_category('misc',top,end,misc2)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df


#Function to get keeping and advance goalkeeping data
def get_keeper_data(top,end):
    df1 = frame_for_category('keepers',top,end,keepers)
    df2 = frame_for_category('keepersadv',top,end,keepersadv2)
    df = pd.concat([df1, df2], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df



#Function to get team-wise data accross all categories as mentioned above
def get_team_data(top,end,text):
    df1 = frame_for_category_team('stats',top,end,stats3,text)
    df2 = frame_for_category_team('keepers',top,end,keepers3,text)
    df3 = frame_for_category_team('keepersadv',top,end,keepersadv2,text)
    df4 = frame_for_category_team('shooting',top,end,shooting3,text)
    df5 = frame_for_category_team('passing',top,end,passing2,text)
    df6 = frame_for_category_team('passing_types',top,end,passing_types2,text)
    df7 = frame_for_category_team('gca',top,end,gca2,text)
    df8 = frame_for_category_team('defense',top,end,defense2,text)
    df9 = frame_for_category_team('possession',top,end,possession2,text)
    df10 = frame_for_category_team('misc',top,end,misc2,text)
    df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], axis=1)
    df = df.loc[:,~df.columns.duplicated()]
    return df

In [12]:
df_outfield = get_outfield_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

df_outfield.to_csv('fbref_data/outfield_players.csv', index=False)

df_outfield

IndexError: list index out of range

In [4]:
df_keeper = get_keeper_data('https://fbref.com/en/comps/11/','/Serie-A-Stats')

df_keeper.to_csv('fbref_data/keepers_players.csv', index=False)

df_keeper

[<tbody> <tr><th class="left" data-stat="team" scope="row"><a href="/en/squads/922493f3/Atalanta-Stats">Atalanta</a></th><td class="right" data-stat="players_used">1</td><td class="right group_start" data-stat="gk_games">1</td><td class="right" data-stat="gk_games_starts">1</td><td class="right" csk="90" data-stat="gk_minutes">90</td><td class="right" data-stat="minutes_90s">1.0</td><td class="right iz group_start" data-stat="gk_goals_against">0</td><td class="right iz" data-stat="gk_goals_against_per90">0.00</td><td class="right" data-stat="gk_shots_on_target_against">4</td><td class="right" data-stat="gk_saves">4</td><td class="right" data-stat="gk_save_pct">100.0</td><td class="right" data-stat="gk_wins">1</td><td class="right iz" data-stat="gk_ties">0</td><td class="right iz" data-stat="gk_losses">0</td><td class="right" data-stat="gk_clean_sheets">1</td><td class="right" data-stat="gk_clean_sheets_pct">100.0</td><td class="right iz group_start" data-stat="gk_pens_att">0</td><td cl

[<tbody> <tr><th class="left" data-stat="team" scope="row"><a href="/en/squads/922493f3/Atalanta-Stats">Atalanta</a></th><td class="right" data-stat="players_used">1</td><td class="right" data-stat="minutes_90s">1.0</td><td class="right iz group_start" data-stat="gk_goals_against">0</td><td class="right iz" data-stat="gk_pens_allowed">0</td><td class="right iz" data-stat="gk_free_kick_goals_against">0</td><td class="right iz" data-stat="gk_corner_kick_goals_against">0</td><td class="right iz" data-stat="gk_own_goals_against">0</td><td class="right group_start" data-stat="gk_psxg">0.7</td><td class="right" data-stat="gk_psnpxg_per_shot_on_target_against">0.19</td><td class="right plus" data-stat="gk_psxg_net">+0.7</td><td class="right plus" data-stat="gk_psxg_net_per90">+0.74</td><td class="right group_start" data-stat="gk_passes_completed_launched">4</td><td class="right" data-stat="gk_passes_launched">20</td><td class="right" data-stat="gk_passes_pct_launched">20.0</td><td class="righ

,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,gk_goals_against,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,Michele Di Gregorio,it ITA,GK,Juventus,27-027,1997,1.0,1.0,90.0,0.0,...,24.7,7.0,42.9,37.3,10.0,1.0,10.0,0.0,0.0,4.0
1,Wladimiro Falcone,it ITA,GK,Lecce,29-133,1995,1.0,1.0,90.0,4.0,...,34.5,2.0,50.0,31.5,9.0,0.0,0.0,1.0,1.0,16.0
2,Pierluigi Gollini,it ITA,GK,Genoa,29-158,1995,1.0,1.0,90.0,2.0,...,40.0,7.0,85.7,60.4,17.0,1.0,5.9,0.0,0.0,4.0
3,Jesse Joronen,fi FIN,GK,Venezia,31-155,1993,1.0,1.0,90.0,3.0,...,26.2,7.0,14.3,27.4,21.0,0.0,0.0,1.0,1.0,16.5
4,Mike Maignan,fr FRA,GK,Milan,29-051,1995,1.0,1.0,90.0,2.0,...,24.4,2.0,50.0,41.0,7.0,1.0,14.3,4.0,4.0,22.0
5,Alex Meret,it ITA,GK,Napoli,27-154,1997,1.0,1.0,90.0,3.0,...,21.2,8.0,25.0,24.3,5.0,1.0,20.0,2.0,2.0,21.2
6,Vanja Milinković-Savić,rs SRB,GK,Torino,27-185,1997,1.0,1.0,90.0,2.0,...,34.8,12.0,58.3,55.4,30.0,1.0,3.3,0.0,0.0,9.4
7,Lorenzo Montipò,it ITA,GK,Hellas Verona,28-185,1996,1.0,1.0,90.0,0.0,...,49.8,10.0,100.0,65.4,17.0,0.0,0.0,1.0,1.0,13.5
8,Juan Musso,ar ARG,GK,Atalanta,30-109,1994,1.0,1.0,90.0,0.0,...,33.1,14.0,92.9,56.9,15.0,0.0,0.0,1.0,1.0,17.5
9,Maduka Okoye,ng NGA,GK,Udinese,24-361,1999,1.0,1.0,90.0,1.0,...,45.8,4.0,100.0,65.0,25.0,2.0,8.0,0.0,0.0,5.5


In [3]:
df_team = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats', 'for')

df_team.to_csv('fbref_data/teams.csv', index=False)

df_team

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,Atalanta,16.0,52.0,1.0,11.0,90.0,4.0,2.0,1.0,1.0,...,24.0,10.0,1.0,1.0,0.0,0.0,46.0,20.0,23.0,46.5
1,Bologna,16.0,73.0,1.0,11.0,90.0,1.0,0.0,1.0,1.0,...,17.0,16.0,1.0,1.0,1.0,0.0,44.0,16.0,10.0,61.5
2,Cagliari,14.0,43.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,15.0,7.0,3.0,0.0,0.0,0.0,41.0,11.0,10.0,52.4
3,Como,16.0,43.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,12.0,11.0,1.0,0.0,0.0,0.0,24.0,3.0,12.0,20.0
4,Empoli,16.0,39.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,16.0,18.0,4.0,0.0,0.0,0.0,41.0,16.0,22.0,42.1
5,Fiorentina,16.0,61.0,1.0,11.0,90.0,1.0,0.0,0.0,0.0,...,9.0,11.0,5.0,0.0,0.0,0.0,43.0,11.0,8.0,57.9
6,Genoa,15.0,32.0,1.0,11.0,90.0,2.0,0.0,0.0,1.0,...,16.0,14.0,0.0,0.0,0.0,0.0,27.0,12.0,24.0,33.3
7,Hellas Verona,16.0,33.0,1.0,11.0,90.0,3.0,3.0,0.0,0.0,...,15.0,13.0,2.0,0.0,0.0,0.0,44.0,13.0,23.0,36.1
8,Inter,16.0,68.0,1.0,11.0,90.0,2.0,2.0,0.0,0.0,...,16.0,12.0,6.0,0.0,1.0,0.0,31.0,24.0,12.0,66.7
9,Juventus,14.0,57.0,1.0,11.0,90.0,3.0,2.0,0.0,0.0,...,11.0,12.0,4.0,0.0,0.0,0.0,34.0,12.0,3.0,80.0


In [4]:
df_vsteam = get_team_data('https://fbref.com/en/comps/11/','/Serie-A-Stats', 'vs')

df_vsteam.to_csv('fbref_data/teams_vs.csv', index=False)

df_vsteam

,team,players_used,possession,games,games_starts,minutes,goals,assists,pens_made,pens_att,...,fouls,fouled,offsides,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct
0,vs Atalanta,16.0,48.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,10.0,24.0,1.0,0.0,1.0,0.0,41.0,23.0,20.0,53.5
1,vs Bologna,16.0,27.0,1.0,11.0,90.0,1.0,1.0,0.0,1.0,...,17.0,16.0,1.0,1.0,1.0,0.0,31.0,10.0,16.0,38.5
2,vs Cagliari,14.0,57.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,7.0,15.0,2.0,0.0,0.0,0.0,37.0,10.0,11.0,47.6
3,vs Como,16.0,57.0,1.0,11.0,90.0,3.0,2.0,0.0,0.0,...,11.0,12.0,4.0,0.0,0.0,0.0,34.0,12.0,3.0,80.0
4,vs Empoli,16.0,61.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,19.0,16.0,1.0,0.0,0.0,0.0,30.0,22.0,16.0,57.9
5,vs Fiorentina,16.0,39.0,1.0,11.0,90.0,1.0,1.0,0.0,0.0,...,15.0,9.0,1.0,0.0,0.0,0.0,35.0,8.0,11.0,42.1
6,vs Genoa,15.0,68.0,1.0,11.0,90.0,2.0,2.0,0.0,0.0,...,16.0,12.0,6.0,0.0,1.0,0.0,31.0,24.0,12.0,66.7
7,vs Hellas Verona,16.0,67.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,13.0,15.0,0.0,0.0,0.0,0.0,39.0,23.0,13.0,63.9
8,vs Inter,16.0,32.0,1.0,11.0,90.0,2.0,0.0,0.0,1.0,...,16.0,14.0,0.0,0.0,0.0,0.0,27.0,12.0,24.0,33.3
9,vs Juventus,14.0,43.0,1.0,11.0,90.0,0.0,0.0,0.0,0.0,...,12.0,11.0,1.0,0.0,0.0,0.0,24.0,3.0,12.0,20.0


In [ ]:
print('Outfield player columns')
print(df_outfield.columns.values)

print('Keeper player columns')
print(df_keeper.columns.values)

print('Team columns')
print(df_team.columns.values)

print('Vs Team columns')
print(df_team.columns.values)

Scrape data for past seasons. 

Do not repeat if data is already present.

In [15]:
import time
old_season = ['2023-2024']
#old_season = ['2022-2023']

for s in old_season:
    comp = 'https://fbref.com/en/comps/11/' + s + '/'
    statstr = '/' + s + '-Serie-A-Stats'
    
    season_short = s[2:4] + s[7:9]
    
    df_outfield = get_outfield_data(comp, statstr)
    df_outfield.to_csv('fbref_data/season' + season_short + '/outfield_players.csv', index=False)
    time.sleep(60)
    
    df_keeper = get_keeper_data(comp, statstr)
    df_keeper.to_csv('fbref_data/season' + season_short + '/keepers_players.csv', index=False)
    time.sleep(60)
    
    df_team = get_team_data(comp, statstr, 'for')
    df_team.to_csv('fbref_data/season' + season_short + '/teams.csv', index=False)
    time.sleep(60)
    
    df_vsteam = get_team_data(comp, statstr, 'vs')
    df_vsteam.to_csv('fbref_data/season' + season_short + '/teams_vs.csv', index=False)
    time.sleep(60)
